In [1]:
from findmycells.main import Project
import os
import pandas as pd
from datetime import datetime

In [2]:
if os.path.isdir('/mnt/c/Users/dsege/TEMP/test_project2/'):
    project_root_dir = '/mnt/c/Users/dsege/TEMP/test_project2/'
    from findmycells.preprocessing import CropStitchingArtefacts
    from findmycells.segmentation import Deepflash2BinaryAndInstanceSegmentationStrategy
    from findmycells.quantifications import ReconstructCellsIn3DFrom2DInstanceLabels, CountCellsInWholeStructure
    from findmycells.inspection import InspectReconstructedCellsBasedOnMultiMatchIDX
else:
    project_root_dir = '/Users/Sofie/Documents/DCL/for_sofie/'

In [3]:
user_input_via_gui = {'project_root_dir': project_root_dir}
os.listdir(project_root_dir)

['02_preprocessed_images',
 '06_inspection',
 '.DS_Store',
 '05_instance_segmentations',
 '04_binary_segmentations',
 '01_rois_to_analyze',
 '07_results',
 '00_microscopy_images',
 '03_deepflash2']

In [4]:
test_project = Project(user_input_via_gui)

In [5]:
test_project.load_status()

In [6]:
test_project.save_status()

### Option A: start a new project

In [3]:
user_input_via_gui = {'project_root_dir': project_root_dir,
                      'preprocessing_configs': {0: {'method_category': None,
                                                    'method_specifier': None,
                                                    'additional_settings': dict(),
                                                    'ProcessingMethod': CropStitchingArtefacts(),
                                                    'ProcessingStrategy': None}
                                               },
                     'segmentation_strategy': Deepflash2BinaryAndInstanceSegmentationStrategy(),
                     'quantification_configs': {'segmentations_to_use': 'instance', # or 'binary'
                                                'quantification_preprocessing_strategies': [ReconstructCellsIn3DFrom2DInstanceLabels()],
                                                'quantification_strategy': CountCellsInWholeStructure(),
                                                'additional_settings': {'min_roi_size': 400,
                                                                        'min_z_extension': 2}},
                     'low_memory': True}

In [4]:
test_project = Project(user_input_via_gui)

### Option b: load status from an existing project:

In [5]:
#test_project = Project({'project_root_dir': project_root_dir})

In [5]:
test_project.load_status()

In [6]:
test_project.database.file_infos['quantification_completed']

[True, True, True, True]

In [9]:
test_project.run_quantificatons()

In [7]:
test_project.database.quantification_results

{'0000': {'000': {'inclusion_criteria_position': ['within', 'intersects'],
   'inclusion_criteria_min_z_extension': 2,
   'number_connected_components': 81}},
 '0001': {'000': {'inclusion_criteria_position': ['within', 'intersects'],
   'inclusion_criteria_min_z_extension': 2,
   'number_connected_components': 19}},
 '0002': {'000': {'inclusion_criteria_position': ['within', 'intersects'],
   'inclusion_criteria_min_z_extension': 2,
   'number_connected_components': 77}},
 '0003': {'000': {'inclusion_criteria_position': ['within', 'intersects'],
   'inclusion_criteria_min_z_extension': 2,
   'number_connected_components': 131}}}

In [14]:
test_project.database.multi_matches_traceback['0000']['original_instance_label_id'][22]

26

In [ ]:
test_project.run_inspection('0000', InspectReconstructedCellsBasedOnMultiMatchIDX(22, save=True, show=True))

In [16]:
test_project.save_status()

# This is where we want to go:

In [ ]:
## Initialize the project

project_name = Project(project_root_dir)
# project_root_dir: path of project root directory

project_name.initialize_database()
# creates the database object and sets-up everything (main subdirs, file infos, ..)
# as alternative, see: load_status()

project_name.save_current_status()
# saves current progress status to disk (basically the database.file_infos as pickle file? - what about shapely polygons that might have been loaded?)


project_name.load_status()
# Alternative to initialize_database()
# loads the corresponding information from disk and allows the user to continue from here


## Preprocessing

project_name.preprocess(microscopy_files, cropping_strategy)
# microscopy_files: list of all microscopy image files that shall be preprocessed (this would allow step-by-step processing)
# cropping_strategy: object of type CroppingStrategy that specifies which exact cropping strategy shall be used

# infers:
    # microscopy-image-file-loader from filetype (e.g. .czi file loader)
    # whether it´s a z-stack
    # whether it´s an RGB image
# automatically updates the database accordingly


## Image segmentation

project_name.run_segmentations(files, SegmentationStrategy)
# files: list of files that shall be segmented (in case of df2 make sure that all are used due to image statistics & cellpose diameter)
# SegmentationStrategy: e.g. deepflash2_binary, deepflash2_instances, intellesis

# launches the respective segmentations and saves all progress in the database object & all segmentations in the corresponding subdir


## Quantifications

project_name.run_quantifications(files, QuantificationStrategy)
# files: list of files whose segmentations shall be used for quantification
# QuantificationStrategy: e.g. 2D, 3D_grid_estimation, 3D_entire_volume (enable selection of more specific options / exclusion criteria)

# Launches the respective quantifications and saves all progress in the database object & all quantification results in the results table